# Multinomial Naive Bayes Classifier for Hate Speech Detection

### Import Libraries

In [6]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from evaluate_classification import EvaluateBinaryClassification

### Initialise Random variables

In [2]:
SEED = 123
np.random.seed(SEED)

### Loading Datam

In [3]:
BASE = 'D:\\ResearchDataGtx1060\\SentimentData\\Hate\\'
fins_train = ['random_hate_train.csv']
fins_test = ['eastasian_hate_test.csv']
track = 0

In [4]:
# We apply only this preprocessing because our data is already preprocessed
def cleanNonAscii(text):
    '''
    Remove Non ASCII characters from the dataset.
    Arguments:
        text: str
    returns: 
        text: str
    '''
    return ''.join(i for i in text if ord(i) < 128)

In [7]:
df_train = pd.read_csv(BASE+fins_train[track])
df_train.head()

,label,text
0,1,<user> if you are one of the <number> mil <has...
1,0,best <hashtag> law of attraction </hashtag> <h...
2,1,<hashtag> michelle obama </hashtag> is the mos...
3,0,smiling because life is good rite now ! <repea...
4,0,ã ¢ â  â ¤ ã ¯ â ¸ â  ã ¢ â  â ¤ ã ¯ â ¸ â ...


In [8]:
df_train.groupby('label').count()

,text
label,
0,2242
1,2242


In [9]:
df_train['text'] = df_train['text'].apply(cleanNonAscii)
df_train.head(5)

,label,text
0,1,<user> if you are one of the <number> mil <has...
1,0,best <hashtag> law of attraction </hashtag> <h...
2,1,<hashtag> michelle obama </hashtag> is the mos...
3,0,smiling because life is good rite now ! <repea...
4,0,<hashtag> ...


In [10]:
X_train, y_train = df_train['text'].values, df_train['label'].values

In [11]:
df_test = pd.read_csv(BASE+fins_test[track])
df_test.head()

,label,text
0,1,<user> <user> the chinese are probably sprayin...
1,0,rt <user> : unpatriotic losers are tweeting ou...
2,1,<user> thus <hashtag> 2019 n co v </hashtag> i...
3,0,north korea closes borders to avoid coronaviru...
4,1,<user> this is a declaration of war . it prove...


In [12]:
df_test.groupby('label').count()

,text
label,
0,3898
1,3898


In [13]:
df_test['text'] = df_test['text'].apply(cleanNonAscii)
df_test.head(5)

,label,text
0,1,<user> <user> the chinese are probably sprayin...
1,0,rt <user> : unpatriotic losers are tweeting ou...
2,1,<user> thus <hashtag> 2019 n co v </hashtag> i...
3,0,north korea closes borders to avoid coronaviru...
4,1,<user> this is a declaration of war . it prove...


In [14]:
X_test, y_test = df_test['text'].values, df_test['label'].values

### Transforming data suitable for model format

In [15]:
count_vectorizer = CountVectorizer(binary=False)
count_vectorizer.fit(X_train)
#count_vectorizer.vocabulary_

CountVectorizer()

In [16]:
train_vectors = count_vectorizer.transform(X_train)
X_train_vectors = train_vectors.toarray()
test_vectors = count_vectorizer.transform(X_test)
X_test_vectors = test_vectors.toarray()

### define and fit the model

In [17]:
param_grid = {
                 'alpha': (0.03, .7, .8)
             }

In [19]:
clf_MultinomialNB = MultinomialNB()
grid_nbc = GridSearchCV(clf_MultinomialNB, param_grid, cv=10)
grid_nbc.fit(X_train_vectors, y_train)

GridSearchCV(cv=10, estimator=MultinomialNB(),
             param_grid={'alpha': (0.03, 0.7, 0.8)})

In [24]:
print("Best parameters set found on development set:")
print(grid_nbc.best_params_)

Best parameters set found on development set:
{'alpha': 0.8}


### Evaluating the model with test dataset

In [21]:
actual, predicted = y_test, grid_nbc.predict(X_test_vectors)

In [22]:
ebc = EvaluateBinaryClassification(gnd_truths = actual, predictions = predicted)
print(ebc.get_full_report())

EvaluateBinaryClassification Object Created

Total Samples	7796
Positive Samples	3898
Negative Samples	3898
True Positive	2851
True Negative	1820
False Positive	2078
False Negative	1047
Accuracy	0.599153412006157
Precision	0.5784134712923514
Recall	0.7314007183170856
F1 Measure	0.645972584116914
Cohen Kappa Score	0.19830682401231403
Area Under Curve	0.5991534120061569

              precision    recall  f1-score   support

           0       0.63      0.47      0.54      3898
           1       0.58      0.73      0.65      3898

    accuracy                           0.60      7796
   macro avg       0.61      0.60      0.59      7796
weighted avg       0.61      0.60      0.59      7796



In [23]:
ebc.save_full_report(model_name='MNB', path='C:\\Users\\User\\JupyterPythonPredator\\COVID19\\domain_adaptation_rerun_randomhate_eastasianhate_')